In [27]:
import os
import json
import itertools
from domain_groups import group_domains
from collections import defaultdict, Counter
DIR = "../optimization/"

DOMAINS = [
    "barman", "blocksworld", "childsnack", "data-network", "depots", "driverlog",
    "elevators", "floortile", "grid", "gripper", "hiking", "logistics",
    "miconic", "nomystery", "openstacks", "parking", "rovers", "satellite",
    "scanalyzer", "snake", "storage", "tpp", "transport", "visitall",
    "woodworking", "zenotravel",
]

TRAINING_PLANNERS_OPT = [
    "fd1906-blind",
    "fd1906-fdss1-mas1-60s",
    "fd1906-fdss1-mas2-60s",
    "fd1906-bjolp",
    "fd1906-lmcut",
    "ipc2014-opt-symba1",
]
EVALUATION_PLANNERS_OPT = [
    "ipc2018-opt-complementary2-3584mb",
    "ipc2018-opt-decstar",
    "ipc2018-opt-delfi-blind",
    "ipc2018-opt-delfi-celmcut",
    "ipc2018-opt-delfi-ipdb",
    "ipc2018-opt-delfi-mas-miasm",
    "ipc2018-opt-delfi-mas-sccdfp",
    "ipc2018-opt-scorpion",
]

TRAINING_PLANNERS_SAT = [
    "fd1906-gbfs-ff",
    "fd1906-lama-first",
    "ipc2014-agl-mpc",
    "ipc2014-agl-mercury",
    "ipc2014-agl-jasper",
    "ipc2014-agl-probe",
]
EVALUATION_PLANNERS_SAT = [
    "ipc2018-agl-decstar",
    "ipc2018-agl-fd-remix",
    "ipc2018-agl-lapkt-dual-bfws",
    "ipc2018-agl-lapkt-bfws-pref",
    "ipc2018-agl-lapkt-poly-bfws",
    "ipc2018-agl-olcff",
    "ipc2018-agl-saarplan",
    "ipc2018-agl-cerberus",
]

TRAINING_PLANNERS = {
        "opt-ipc" : TRAINING_PLANNERS_OPT,
        "opt-1204": TRAINING_PLANNERS_OPT,
        "sat-ipc": TRAINING_PLANNERS_SAT,
        "sat-1204": TRAINING_PLANNERS_SAT,
}

EVALUATION_PLANNERS = {
        "opt-ipc" : EVALUATION_PLANNERS_OPT,
        "opt-1204": EVALUATION_PLANNERS_OPT,
        "sat-ipc": EVALUATION_PLANNERS_SAT,
        "sat-1204": EVALUATION_PLANNERS_SAT,
}

FILENAMES = {
        "opt-ipc": ["2020-11-23-A-optimization-planners-ipc-properties.json", "2020-12-05-A-evaluation-opt-ipc-properties.json"],
        "opt-1204": ["2020-12-05-C-evaluation-opt-new2014-properties.json"],
        "sat-ipc": ["2020-11-23-D-optimization-planners-sat-ipc-properties.json","2020-12-05-B-evaluation-sat-ipc-properties.json"],
        "sat-1204": ["2020-12-05-D-evaluation-sat-new2014-properties.json"]
    }

def read_runs(filename):
    with open(filename) as f:
        data = json.load(f)
        for run, values in data.items(): 
            if "coverage" not in values: 
                #print ("Warning, coverage attribute missing in ", filename, run)
                pass
        return data.values()

# Step 1: gather all runs and label them by dataset and domain
all_runs = defaultdict(list)
for name, filenames in FILENAMES.items():
    
    for filename in filenames: 
        new_runs = read_runs(os.path.join(DIR, "results", filename))
        new_runs = [group_domains(run) for run in new_runs]
        new_runs = [run for run in new_runs if run["domain"] in DOMAINS]
        
        for run in new_runs: 
            run["dataset"] = name
            all_runs[(name, run["domain"])].append(run)
        
        

In [28]:
def compute_coverage (properties, atr_name, runs, time_limit=100000):
    coverage = Counter()
    for run in runs: 
        if "coverage" in run and run["coverage"] == 1 and run["planner_wall_clock_time"] <= time_limit: 
            coverage[run["algorithm"]] += 1
    properties[atr_name] = coverage
    

def compute_comparisons (properties, comp_atr_name, cov_atr_name, planners):
    coverage = properties[cov_atr_name]
    properties[comp_atr_name] = sum ([1 for (p1, p2) in itertools.combinations(planners, 2) if coverage[p1] != coverage[p2]])
    

# Step 2: Compute all aggregated properties for a dataset
properties_dataset = defaultdict(dict)
for dataset, domain in all_runs: 
    if "-ipc" in dataset: 
        continue
    ipcdataset = dataset.split("-")[0] + "-ipc"
    
    properties_dataset[(dataset, domain)] ["track"] = dataset
    properties_dataset[(dataset, domain)] ["domain"] = domain
    compute_coverage(properties_dataset[(dataset, domain)], "coverage", all_runs[(dataset, domain)])
    compute_coverage(properties_dataset[(dataset, domain)], "coverage30s", all_runs[(dataset, domain)], 30)
    compute_coverage(properties_dataset[(dataset, domain)], "coverage300s", all_runs[(dataset, domain)], 300)
    
    compute_coverage(properties_dataset[(dataset, domain)], "coverage-ipc", all_runs[(ipcdataset, domain)])
    compute_coverage(properties_dataset[(dataset, domain)], "coverage30s-ipc", all_runs[(ipcdataset, domain)], 30)
    compute_coverage(properties_dataset[(dataset, domain)], "coverage300s-ipc", all_runs[(ipcdataset, domain)], 300)

    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons-training", "coverage", TRAINING_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons-eval", "coverage", EVALUATION_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons30s-training", "coverage30s", TRAINING_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons30s-eval", "coverage30s", EVALUATION_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons300s-training", "coverage300s", TRAINING_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons300s-eval", "coverage300s", EVALUATION_PLANNERS[dataset])

    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons-training-ipc", "coverage-ipc", TRAINING_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons-eval-ipc", "coverage-ipc", EVALUATION_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons30s-training-ipc", "coverage30s-ipc", TRAINING_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons30s-eval-ipc", "coverage30s-ipc", EVALUATION_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons300s-training-ipc", "coverage300s-ipc", TRAINING_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons300s-eval-ipc", "coverage300s-ipc", EVALUATION_PLANNERS[dataset])
    
    for x in ["comparisons-training", "comparisons-eval", "comparisons300s-training", "comparisons300s-eval" ]:
        properties_dataset[(dataset, domain)][f"{x}-ipcdiff"] = properties_dataset[(dataset, domain)][x] - properties_dataset[(dataset, domain)][f"{x}-ipc"] 
    


In [29]:
LOGDIR = "../logfiles/2020-12-04/"
import os
import re

def parse_CPLEX_log(content):
    regex_int = [re.compile(x) for x in [".*Different evaluated sequences: (?P<evaluated_sequences>(\d+))",
                                     ".*Candidate sequences: (?P<candidate_sequences>(\d+))", 
                                    ]]
    regex_float = [re.compile(x) for x in [".*Solution value  = (?P<solution_value>(.*))",]]

    penalties = [float(l.split(":")[-1]) for l in content if l.startswith("Penalty: ")]
        
    data = {
            "sart_runtimes" : False, 
            "num_sequences" : 0, 
            "min_penalty" : min(penalties) if penalties else "-", 
            "max_penalty" : max(penalties) if penalties else "-",  
           }
            
    for l in content: 
        if "Using sart runtimes on CPLEX optimization" in l: 
            data["sart_runtimes"] = True
            continue
        if "Selected: sequence" in l:
            data["num_sequences"] += 1

            
        for reg in regex_int: 
            if reg.match(l): 
                for atr, value in reg.match(l).groupdict().items():
                    data [atr] = int(value)
        
        for reg in regex_float: 
            if reg.match(l): 
                for atr, value in reg.match(l).groupdict().items():
                    data [atr] = float(value)
    return data
    
for logfile in sorted(os.listdir(LOGDIR)):
    _, track, domain = logfile.split("_")
    
    dataset = track.replace("14", "-1204")
    
    f = open(f"{LOGDIR}/{logfile}")
    content = f.readlines()
    
    properties_dataset[(dataset, domain)] ["track"] = dataset
    properties_dataset[(dataset, domain)] ["domain"] = domain

    data = parse_CPLEX_log(content)
    properties_dataset[(dataset, domain)].update(data)
    



                 

In [31]:
import pandas as pd

from IPython.display import display

def make_short(x): 
    return x.replace("comparisons", "cmp").replace("training", "tr")
ATRIBUTES_TABLE = ["track", "domain", "solution_value", "sart_runtimes", "comparisons-training", "comparisons-training-ipcdiff", "comparisons-eval", "comparisons-eval-ipcdiff"]

table = []
for data in properties_dataset.values(): 
    table.append([data[atr] if atr in data else "-" for atr in ATRIBUTES_TABLE ])


pd.options.display.float_format = "{:,.2g}".format
pd.set_option('display.max_rows', 500)

df = pd.DataFrame(table,columns =  list(map(make_short, ATRIBUTES_TABLE)))
display(df)




            

,track,domain,solution_value,sart_runtimes,cmp-tr,cmp-tr-ipcdiff,cmp-eval,cmp-eval-ipcdiff
0,opt-1204,barman,60,True,5,0,24,12
1,opt-1204,blocksworld,6.9e+09,True,14,2,24,6
2,opt-1204,childsnack,43,True,5,0,18,5
3,opt-1204,data-network,33,True,14,-1,26,-1
4,opt-1204,depots,40,True,13,4,25,0
5,opt-1204,driverlog,19,True,14,3,26,4
6,opt-1204,elevators,46,True,14,-1,23,-3
7,opt-1204,floortile,16,True,13,0,25,4
8,opt-1204,grid,12,True,14,3,26,7
9,opt-1204,gripper,7.9e+08,True,11,0,7,0
